### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### 1.  Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [46]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
## copy your path in here:
# Path Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [47]:
# read in 2005 data for the reciprocity measures ##### positive not renamed yet
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12604", "vp12601", "vp12606"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12601': 'take_revenge', 'vp12604': 'similar_problems', 'vp12606': 'insult_back'})
df_05.head(3)

,,syear,similar_problems,take_revenge,insult_back
pid,hid,,,,
201,27,2005,[3] Skala 1-7,[6] Skala 1-7,[4] Skala 1-7
203,60313,2005,[4] Skala 1-7,[7] Trifft voll zu,[5] Skala 1-7
602,60,2005,[5] Skala 1-7,[5] Skala 1-7,[5] Skala 1-7


In [48]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay'})

In [49]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001' ,  'xp5701' , 'xp0102']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job',  'xp5701' : 'commute_distance' , "xp0102" : "work_satisfaction"})

In [50]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil", "betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})

 ### 2. Define Functions and mappings for cleaning data

In [51]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 1,
    '[2] Nein': 0,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
#spelling error
take_revenge_mapping = {
    '[1] Trifft ueberhautp nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}

# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for job satisfaction: split up into binary with roughly equal value counts for simplicity: might change that later to categories,
# vielleicht mapping nochmal ändern ? 
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention robustness check: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}

## function for recoding values and dropping missing

def recode_categoricals(inputdf):
    """
        merges data from different years, applies recoding to categoricals and constructs additional variables.

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            - rc_rec_binary: optional argument: if == 1 neg. reciprocity will be split into 2 groups: high and low
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","insult_back"]] = merged[["similar_problems","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    merged["take_revenge"] = merged["take_revenge"].map(take_revenge_mapping)
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode job satisfaction
    merged['work_satisfaction']= merged['work_satisfaction'].map(satisfaction_mapping)
    # recode turnover intention variable
    merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen

def merge_and_clean(df_05,df_06,df_07,work07):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df).astype('int')

    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan) 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    
    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'work_satisfaction'])
    # Convert 'gender' and 'sector' columns to categorical data type
    for col in ['gender']:
        df_mincer[col] = df_mincer[col].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

### Inlcude People who switched their jobs in 2006-2007 with 2006 controls

In [52]:
def include_jobchangers(data07,onlynewemployer=True):
    """
    Input: 
            df: finished 2007 dataframe
            onlynewemployer: optional argument: If True only includes those who switched jobs to a new employer
            worksatisfaction: if True drops observations where job satisfaction between 2006 and 2007 changed to check for our Assumption
    
    """
    df = data07
    # read in cleaned 2006 dataset:
    #  change path here : Maxie: '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    #                     Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'  
    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchangepositiverec.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # If cardinal turnover intentions are coded to 100
    # drop s year columns from both dataframes
    df.drop(columns=df.filter(regex='^syear').columns, inplace=True)
    
    # create list of IDs of people who switched to a new employer in the last year in 2007
    if onlynewemployer == True:
        IDs_tokeep = df[(df["new_job"] == 1) & (df["reason_new_job"] == '[3] Stelle bei neuen Arbeitgeber')].index
    # if True drops people whose work satisfaction changed between 2006 and 2007
    else:
        IDs_tokeep = df[(df["new_job"] == 1)].index
    
    # drop all who changed their job in 2007 dataframe and replace 2007 controls with 2006 controls
    df.drop(df[df["new_job"] == 1].index, inplace = True)
    observations_2006.drop(observations_2006[~observations_2006.index.isin(IDs_tokeep)].index, inplace=True)
    #concat both dataframes
    dfconcat = pd.concat([df,observations_2006])
    
    return dfconcat

### 3. Analysis
#### 1.Probit Regression

- Probit Estimation: TurnoverIntention_{2005} = Constant + Neg-Rec + Unfair + Rec X Unfair + Controls + Error

- As the measure for unfair treatment in the first regression i first used recog_effort -> decide for one later
    - "When I consider all my accomplishments and efforts, the recognition of I've received seems about right to me"

In [53]:
df_OLS = add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07))
df_OLS.drop(columns=df_OLS.filter(regex='^syear').columns, inplace=True)
df_OLS.drop(df_OLS[df_OLS["new_job"] == 1].index, inplace = True)
# specify columns which we need for regression by dropping everything else
df_OLS = df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup','wage_lastmonth','work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month
# add interaction term
df_OLS["recXrecog_effort"] = df_OLS["recog_effort"] * df_OLS["avg_rec"]
# drop missing data in regression dataframe
df_OLS.dropna(inplace=True)

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-51-047203a7ccc4>:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mincer[col] = df_mincer[col].astype('category')


In [54]:
from statsmodels.discrete.discrete_model import Probit
# Define Outcome variable
Y = df_OLS["turnover_intention"]
# define X matrix
X = df_OLS.drop(columns=["turnover_intention"])
# add constant 
X = sm.add_constant(X)
# build model
model = Probit(Y, X.astype(float))
# estimate model
probit_model = model.fit()
# print summary table
probit_model.summary()

         Current function value: 0.592775
         Iterations: 35


c:\Anaconda\anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:     turnover_intention   No. Observations:                 5409
Model:                         Probit   Df Residuals:                     5395
Method:                           MLE   Df Model:                           13
Date:                Wed, 11 Jan 2023   Pseudo R-squ.:                  0.1444
Time:                        20:55:47   Log-Likelihood:                -3206.3
converged:                      False   LL-Null:                       -3747.6
Covariance Type:            nonrobust   LLR p-value:                3.202e-223
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0592   4.04e+05   1.46e-07      1.000   -7.93e+05    7.93e+05
recog_effort            -0.2864      0.257     -1.116      0.264      -0.789       0.217
working_hours           -0.0019      0.003     -0.745      0.456      -0.007       0.003
firmsize                -0.0353      0.007     -5.387      0.000      -0.048      -0.022
tenure                  -0.0262      0.002    -11.002      0.000      -0.031      -0.022
years_educ               0.0565      0.007      8.216      0.000       0.043       0.070
gender                   0.1617      0.043      3.751      0.000       0.077       0.246
commute_distance         0.0012      0.000      2.924      0.003       0.000       0.002
avg_rec                 -0.0819      0.034     -2.391      0.017      -0.149      -0.015
age                      0.0762   4.49e+04    1.7e-06      1.000   -8.81e+04    8.81e+04
potential_experience     0.0006   1248.070   4.57e-07      1.000   -2446.171    2446.172
age_squared             -0.1649   1.25e+05  -1.32e-06      1.000   -2.45e+05    2.45e+05
mincer_residuals     -4.271e-05   2.79e-05     -1.530      0.126   -9.74e-05     1.2e-05
recXrecog_effort        -0.0075      0.043     -0.175      0.861      -0.092       0.077
========================================================================================
"""

#### 2. OLS
Now, instead of using a probit model, we use OLS.  linear probability model

In [55]:
from statsmodels.regression.linear_model import OLS
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")
lpm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     577.7
Date:                Wed, 11 Jan 2023   Prob (F-statistic):               0.00
Time:                        20:55:48   Log-Likelihood:                -3384.5
No. Observations:                5409   AIC:                             6795.
Df Residuals:                    5396   BIC:                             6881.
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0805      0.010      8.073      0.000       0.061       0.100
recog_effort            -0.1032      0.088     -1.176      0.240      -0.275       0.069
working_hours           -0.0009      0.001     -1.038      0.299      -0.003       0.001
firmsize                -0.0120      0.002     -5.304      0.000      -0.016      -0.008
tenure                  -0.0086      0.001    -11.366      0.000      -0.010      -0.007
years_educ               0.0195      0.002      8.875      0.000       0.015       0.024
gender                   0.0576      0.014      3.986      0.000       0.029       0.086
commute_distance         0.0004      0.000      3.214      0.001       0.000       0.001
avg_rec                 -0.0285      0.012     -2.405      0.016      -0.052      -0.005
age                      0.0847      0.009      9.243      0.000       0.067       0.103
potential_experience     0.0020      0.000      6.319      0.000       0.001       0.003
age_squared             -0.2302      0.029     -7.924      0.000      -0.287      -0.173
mincer_residuals     -1.331e-05    9.4e-06     -1.416      0.157   -3.17e-05    5.12e-06
recXrecog_effort        -0.0017      0.015     -0.114      0.909      -0.031       0.027
==============================================================================
Omnibus:                   119566.035   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              413.435
Skew:                          -0.039   Prob(JB):                     1.67e-90
Kurtosis:                       1.648   Cond. No.                     6.64e+17
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 1.12e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""